In [155]:
import pandas as pd
import random
import numpy as np

# Durasi kegiatan
durations = {
    "A": 2,
    "B": 3,
    "C": 2,
    "D": 4
}

# Batas total durasi
total_duration = 8

# Syarat:
# 1. A harus sebelum B.
# 2. C dan D tidak boleh bersamaan.
# 3. Total durasi tidak boleh melebihi 8 jam.

# Representasi kromosom: [A_start, B_start, C_start, D_start]
# Durasi dihitung berdasarkan waktu mulai masing-masing kegiatan.
def fitness(schedule):
    A_start, B_start, C_start, D_start = schedule

    # Durasi masing-masing kegiatan
    A_end = A_start + durations["A"]
    B_end = B_start + durations["B"]
    C_end = C_start + durations["C"]
    D_end = D_start + durations["D"]

    # Syarat 1: A sebelum B
    condition1 = A_end <= B_start

    # Syarat 2: C dan D tidak bersamaan
    condition2 = not (
        (C_start < D_end and C_end > D_start)
    )

    # Syarat 3: Total durasi tidak melebihi batas
    condition3 = max(A_end, B_end, C_end, D_end) <= total_duration

    # Evaluasi fitness
    if condition1 and condition2 and condition3:
        return 1 / (1 + max(A_end, B_end, C_end, D_end))  # Optimalkan jadwal dengan durasi pendek
    else:
        return 0

# Inisialisasi populasi
def initialize_population(size):
    population = []
    for _ in range(size):
        individual = [
            random.randint(0, total_duration - durations["A"]),
            random.randint(0, total_duration - durations["B"]),
            random.randint(0, total_duration - durations["C"]),
            random.randint(0, total_duration - durations["D"]),
        ]
        population.append(individual)
    return population

# Seleksi orang tua
def select_parents(population, fitness_scores):
    if np.sum(fitness_scores) == 0:  # Cek jika semua nilai fitness_scores nol
        probabilities = np.ones(len(population)) / len(population)  # Probabilitas seragam
    else:
        probabilities = fitness_scores / np.sum(fitness_scores)  # Normalisasi probabilitas

    indices = np.random.choice(
        len(population), size=2, replace=False, p=probabilities
    )
    return population[indices[0]], population[indices[1]]

# Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutasi
def mutate(individual, mutation_rate=0.1):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random.randint(0, total_duration - durations[list(durations.keys())[i]])

# Algoritma Genetika
def genetic_algorithm(generations, population_size, mutation_rate):
    population = initialize_population(population_size)

    for generation in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])

        if np.max(fitness_scores) == 1:
            break  # Solusi terbaik ditemukan

        new_population = []

        while len(new_population) < population_size:
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)

            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)

            new_population.append(child1)
            new_population.append(child2)

        population = new_population[:population_size]

    # Cari solusi terbaik
    best_index = np.argmax([fitness(ind) for ind in population])
    return population[best_index]

# Jalankan GA
best_schedule = genetic_algorithm(generations=100, population_size=50, mutation_rate=0.1)

# Cetak hasil
schedule_result = {
    "Activity": ["A", "B", "C", "D"],
    "Start Time": [best_schedule[0], best_schedule[1], best_schedule[2], best_schedule[3]],
    "End Time": [
        best_schedule[0] + durations["A"],
        best_schedule[1] + durations["B"],
        best_schedule[2] + durations["C"],
        best_schedule[3] + durations["D"],
    ],
    "Duration": [durations["A"], durations["B"], durations["C"], durations["D"]],
}

# Tambahkan kolom total durasi
df_schedule = pd.DataFrame(schedule_result)

# Cetak tabel
print("\nLaporan Jadwal Aktivitas:\n")
print(df_schedule.to_string(index=False))



Laporan Jadwal Aktivitas:

Activity  Start Time  End Time  Duration
       A           0         2         2
       B           4         7         3
       C           5         7         2
       D           0         4         4
